### Entegra Mikro Price Analysis

In [1]:
import unicodedata
import datetime as dt
import os
import sys
import re
import numpy as np
import pandas as pd
from pathlib import Path
import nltk
from nltk import word_tokenize 
import shutil
from os.path import isfile, join
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import locale
from locale import atof
from geopy.geocoders import Nominatim
from geopy.point import Point
from geopy.exc import GeocoderTimedOut
from functools import reduce
from datetime import date

#### Entegra Mikro Data

In [2]:
output_path = r"C:\Users\user\Downloads"

In [4]:
df_entegra_master = pd.read_excel(r"C:\Users\user\Desktop\Data Analysis\Adoba\Sales_Report\Result\Satış Ve Pazarlama\Entegra\Entegra Ürün Satış Ve İptal Adet Ve TL Master (01.01.2024-11.07.2024).xlsx")
df_entegra_master

,product_name,model,toplam_satış_adet,toplam_iptal_adet,toplam_net_satış_tl,toplam_vergi_tl,toplam_brüt_satış_tl
0,"Portatif Katlanabilir Masa, Piknik Masası, Bal...",002583,8357,267,1.231818e+06,126659.4270,1.358478e+06
1,Alas Çok Amaçlı Baza Kutusu Bej Çizgili Tekli ...,002941,8152,239,8.397484e+05,82587.3949,9.223358e+05
2,"Pikas Ahşap Özel İsim Baskılı, Oval Kedi Köpek...",001602,6456,198,8.249409e+05,161310.9999,9.862488e+05
3,"Ahşap Özel İsim Baskılı, Kare Kedi Köpek Mama ...",001603,5195,123,6.677695e+05,132072.2355,7.998412e+05
4,Awox Orion 1800 W Tost Makinesi Siyah Awox002,003722,4678,137,2.144530e+06,421409.0798,2.565939e+06
...,...,...,...,...,...,...,...
1030,"Ahşap Boyama Hayvan Figür Oyuncak Fil, Ayı, Ki...",002558,1,0,1.754200e+02,35.0800,2.105000e+02
1031,"Ahşap Boyama Hayvan Figür Oyuncak Dinazor, Köp...",002557,1,0,1.754167e+02,35.0833,2.105000e+02
1032,"Ahşap Boyama Hayvan Figür Oyuncak At, Kedi, Kö...",002556,1,0,1.754200e+02,35.0800,2.105000e+02
1033,Intex Bardaklı Mavi Kırmızı Yatak 188X71 Cm Id...,003986,1,1,3.333750e+02,66.6750,4.000500e+02


In [5]:
df_entegra_master["model"].nunique()

1035

In [316]:
df_entegra_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1036 entries, 0 to 1035
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   product_name          1036 non-null   object 
 1   model                 1036 non-null   object 
 2   toplam_satış_adet     1036 non-null   int64  
 3   toplam_iptal_adet     1036 non-null   int64  
 4   toplam_net_satış_tl   1036 non-null   float64
 5   toplam_vergi_tl       1036 non-null   float64
 6   toplam_brüt_satış_tl  1036 non-null   float64
dtypes: float64(3), int64(2), object(2)
memory usage: 56.8+ KB


In [317]:
df_mikro_price = pd.read_excel(r"C:\Users\user\Desktop\Data Analysis\Adoba\Sales_Report\Data\Mikro\Product Price\Mikro Product Price (22.07.2024).xlsx")  # ????
df_mikro_price = df_mikro_price.rename(columns={"Ürün Adı":"product_name","Model Kodu":"model","Gelişi":"geliş_fiyat","KDV ORANI":"KDV Oranı"})
df_mikro_price

,product_name,model,Gelişi,KDV Oranı
0,"%100 Doğal At Kılı Selülit Fırçası, Doğal At K...",002972,60.00,20
1,"%100 Doğal Keçi Kılı Yüz Bakım Fırçası, Yüz Ba...",002975,25.00,20
2,%100 Doğal Sarı Kantaron Yağı 50 ml,002960,28.74,20
3,0-3 Yaş Zeka Kartları Dokun Hisset Tanı Eğitim...,002932,118.00,20
4,1 Adet Ayakkabı Hurcu Gri Çizgili,002924,156.00,10
...,...,...,...,...
1404,Şeffaf Kedi Ve Köpek Düz Sırt Taşıma Çantası,002578,66.00,20
1405,Şeffaf Kedi Ve Köpek Düz Sırt Taşıma Çantası Sarı,003865,200.00,20
1406,Şeffaf Kedi ve Köpek Düz Sırt Taşıma Çantası 8 KG,003865,200.00,20
1407,Şişme Tabanlı 3 Boğumlu Sunset Havuz 114x25 cm,004004,145.00,20


In [319]:
df_mikro_price["model"].nunique()

656

In [320]:
df_mikro_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1409 entries, 0 to 1408
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_name  1409 non-null   object 
 1   model         1409 non-null   object 
 2   Gelişi        1409 non-null   float64
 3   KDV Oranı     1409 non-null   int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 44.2+ KB


In [321]:
df_mikro_price.isnull().any()

product_name    False
model           False
Gelişi          False
KDV Oranı       False
dtype: bool

In [322]:
df_mikro_price_duplicated = df_mikro_price[df_mikro_price.duplicated(subset="model", keep=False)]
df_mikro_price_duplicated = df_mikro_price_duplicated.sort_values(by="model")
df_mikro_price_duplicated.reset_index(drop=True, inplace=True)
df_mikro_price_duplicated

,product_name,model,Gelişi,KDV Oranı
0,Akordion Çekmece Iç Organizer - L,001268,42.0,10
1,Çok Amaçlı Çekmece Içi Akordiyon Organizer Gri...,001268,84.0,10
2,Hueva Home 2'li Akordiyon Organizer L,001268,45.0,10
3,Çok Amaçlı Çekmece Içi Akordiyon Organizer Gri...,001269,21.0,10
4,Çok Amaçlı Çekmece Içi Akordiyon Organizer Gri...,001269,45.0,20
...,...,...,...,...
1064,Vıte Pe330 Kıyma Makinesi,004375,2000.0,20
1065,Kadın Kaş Kesme Makası,004380,35.0,20
1066,"Pikas Kadın Kaş Kesme Makası, Taraklı Kaş Makası",004380,35.0,20
1067,"Kedi Köpek 3 lü Set Parmak Diş Fırçası, Siliko...",copy004277,48.5,20


In [323]:
# df_mikro_price_duplicated.to_excel(fr"{output_path}\Mikro Aynı Model Tekrarlama.xlsx", index=False)

#### Check Entegra Mikro Difference

In [ ]:
set_entegra = set(df_entegra_master["model"])
set_mikro = set(df_mikro_price["model"])

In [ ]:
entegra_diff_list = set_entegra.difference(set_mikro)
entegra_diff_list

{'000028',
 '000037',
 '000571',
 '000580',
 '000622',
 '000631',
 '000679',
 '000680',
 '000684',
 '000688',
 '000690',
 '000697',
 '000699',
 '000706',
 '000707',
 '000721',
 '000735',
 '000794',
 '000863',
 '000864',
 '000873',
 '000875',
 '000883',
 '000889',
 '000890',
 '000912',
 '000922',
 '000937',
 '001254',
 '001255',
 '001270',
 '001271',
 '001281',
 '001284',
 '001302',
 '001303',
 '001372',
 '001377',
 '001383',
 '001407',
 '001412',
 '001413',
 '001414',
 '001415',
 '001416',
 '001417',
 '001420',
 '001421',
 '001422',
 '001508',
 '001530',
 '001533',
 '001534',
 '001535',
 '001536',
 '001539',
 '001541',
 '001542',
 '001545',
 '001547',
 '001548',
 '001551',
 '001552',
 '001553',
 '001554',
 '001609',
 '001615',
 '001616',
 '001617',
 '001620',
 '001638',
 '001643',
 '001645',
 '001671',
 '001673Pktm',
 '001708',
 '001718',
 '001724',
 '001775',
 '001870',
 '001874',
 '001875',
 '001877',
 '001878',
 '001880',
 '001882',
 '001883',
 '001884',
 '002037',
 '002053',
 '0020

In [ ]:
df_prod_no_price = df_prod_name[df_prod_name["model"].isin(entegra_diff_list)]
df_prod_no_price

In [ ]:
df_prod_no_price[df_prod_no_price.duplicated(subset="model", keep=False)]

In [ ]:
df_prod_no_price = df_prod_no_price.drop_duplicates(subset="model")
df_prod_no_price.reset_index(drop=True, inplace=True)
df_prod_no_price

In [ ]:
prod_no_price_list = list(df_prod_no_price["model"])
prod_no_price_list

In [ ]:
df_name_var_no_price = df_name_var[df_name_var["Model Kodu"].isin(prod_no_price_list)]
df_name_var_no_price.reset_index(drop=True, inplace=True)
df_name_var_no_price

In [ ]:
# df_name_var_no_price.to_excel(fr"{output_path}\Entegra Fiyat Bilgisi Olmayan.xlsx", index=False)

In [ ]:
set_var_no_price = set(df_name_var_no_price["Model Kodu"])
set_prod_no_price = set(df_prod_no_price["model"])

In [ ]:
list_no_in_mikro_price = list(set_prod_no_price.difference(set_var_no_price))
list_no_in_mikro_price

In [ ]:
df_prod_name_update = pd.read_excel(r"C:\Users\user\Desktop\Data Analysis\Adoba\Sales_Report\Data\Entegra\Product Name\Entegra Model Product Name Update (23.07.2024).xlsx")
df_prod_name_update

,model,product_name
0,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö..."
1,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ..."
2,003954,Awox Barmix Çubuk Blender 1000 W
3,004286,Awox Orbit Gizli Rezistanslı Su Isıtıcı Kettle...
4,002656,Sprey Başlıklı Amber Cam 10 Ml Boş - Yedek Şişe
...,...,...
1193,001265,Çok Amaçlı Çekmece Içi Akordiyon Organizer Gri...
1194,003014,Çömlek Ayaklı Sırlı Seramik Kedi Maması Kabı 2...
1195,gry00018,Özlem Toys Match Up Eşleştirme Kartları 140 Pa...
1196,M-42231,İntex Oyun Havuzu Şişme Taban 100Toplu+Pompa M...


##### Add New Product

In [276]:
df_prod_name = pd.read_excel(r"C:\Users\user\Desktop\Data Analysis\Adoba\Sales_Report\Data\Entegra\Product Name\Entegra Model Product Name.xlsx")
df_prod_name

,model,product_name
0,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö..."
1,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ..."
2,003954,Awox Barmix Çubuk Blender 1000 W
3,004286,Awox Orbit Gizli Rezistanslı Su Isıtıcı Kettle...
4,002656,Sprey Başlıklı Amber Cam 10 Ml Boş - Yedek Şişe
...,...,...
1087,002374,Da69 Ahşap Resim Şovalesi 37 Cm
1088,002421,"Çg65 Ahşap Çocuk Makyaj Masası, Pembe Makyaj M..."
1089,002061,Müller Di̇zel Enjektör Temizleyi̇ci̇ (Üstün Al...
1090,002792,21512-21010 Karter Tapasi Kisa Universal


In [277]:
df_name_var = pd.read_excel(r"C:\Users\user\Downloads\Kopya Ürün Listesi Fiyat Bilgisi.xlsx")
df_name_var_select = df_name_var[["Model Kodu","Ürün Adı"]]
df_name_var_select = df_name_var_select.rename(columns={"Model Kodu":"model","Ürün Adı":"product_name"})
df_name_var_select.drop_duplicates(inplace=True)
df_name_var_select

,model,product_name
0,002972,"%100 Doğal At Kılı Selülit Fırçası, Doğal At K..."
1,002975,"%100 Doğal Keçi Kılı Yüz Bakım Fırçası, Yüz Ba..."
2,002960,%100 Doğal Sarı Kantaron Yağı 50 ml
3,002932,0-3 Yaş Zeka Kartları Dokun Hisset Tanı Eğitim...
4,002924,1 Adet Ayakkabı Hurcu Gri Çizgili
...,...,...
1573,003865,Şeffaf Kedi Ve Köpek Düz Sırt Taşıma Çantası Sarı
1574,003865,Şeffaf Kedi ve Köpek Düz Sırt Taşıma Çantası 8 KG
1575,BAG-21,Şeffaf Tekerlekli Kedi Köpek Taşıma Valizi Çek...
1576,004004,Şişme Tabanlı 3 Boğumlu Sunset Havuz 114x25 cm


In [278]:
df_name_var_select.isnull().any()

model            True
product_name    False
dtype: bool

In [279]:
df_name_var_select.dropna(inplace=True)

In [280]:
df_name_var_select.isnull().any()

model           False
product_name    False
dtype: bool

In [281]:
set_origin = set(df_prod_name["model"])
set_var = set(df_name_var_select["model"])

In [282]:
set_origin.difference(set_var)

{'000028',
 '000037',
 '000571',
 '000580',
 '000622',
 '000631',
 '000680',
 '000690',
 '000697',
 '000706',
 '000707',
 '000721',
 '000735',
 '000794',
 '000801',
 '000863',
 '000864',
 '000873',
 '000875',
 '000883',
 '000889',
 '000890',
 '000937',
 '001254',
 '001255',
 '001270',
 '001271',
 '001281',
 '001284',
 '001302',
 '001303',
 '001377',
 '001383',
 '001407',
 '001414',
 '001416',
 '001417',
 '001420',
 '001421',
 '001422',
 '001508',
 '001530',
 '001533',
 '001534',
 '001535',
 '001541',
 '001542',
 '001545',
 '001547',
 '001548',
 '001552',
 '001553',
 '001609',
 '001617',
 '001638',
 '001643',
 '001645',
 '001656',
 '001671',
 '001673Pktm',
 '001708',
 '001718',
 '001722',
 '001724',
 '001775',
 '001870',
 '001875',
 '001877',
 '001878',
 '001880',
 '001881',
 '001882',
 '001883',
 '001884',
 '002037',
 '002053',
 '002058',
 '002061',
 '002062',
 '002064',
 '002068',
 '002078',
 '002081',
 '002091',
 '002095',
 '002096',
 '002104',
 '002123',
 '002124',
 '002138',
 '0022

In [283]:
new_prod_list = list(set_var.difference(set_origin))
new_prod_list

['002551',
 '003012',
 '003820',
 '001266',
 'Crclbeginner',
 '004362',
 '002919',
 'TYC00519735403',
 '003883',
 '001601pktb',
 'gjhghj13456',
 '003627',
 'CM3437',
 '003601',
 'BAG-21',
 '003822',
 '004132',
 '3837',
 '002539',
 '070620221607',
 '003117',
 'FURKSOAN',
 '003457',
 '004029',
 '004264',
 'dop8262838igo',
 '002987',
 3170,
 '3741',
 '003454',
 '002545pkt',
 '004357',
 'M-42231',
 'FRK.58499',
 '002121',
 '002696',
 '003437',
 '002543',
 4217,
 '002885',
 '970',
 '003458',
 '003442',
 'TYC00677703360',
 '003436',
 '473834',
 '26027',
 'TBH-012721',
 'CRCL-104',
 '003885',
 '3742',
 '004215',
 'copy004229',
 '004048',
 'BF182',
 '001265',
 '3805',
 'BF177',
 '8699329758048',
 '003886',
 'CRCL130',
 '004379',
 'Ahir6529',
 '001287',
 '2016ST023123228553951',
 '002537',
 '003887',
 '003391',
 'BHS-0023',
 'ALS MG GR',
 '003398',
 'Combo Scooter',
 '003821',
 'Hobby Life Orgamix 03-1062',
 'SCT2021022-2',
 '003435',
 '001601',
 '002836',
 '003718',
 4319,
 '003474',
 '001601P

In [284]:
df_new_prod = df_name_var_select[df_name_var_select["model"].isin(new_prod_list)]
df_new_prod

,model,product_name
4,002924,1 Adet Ayakkabı Hurcu Gri Çizgili
10,CRCL130,10’lu Geometrik Yumurta
14,TYBP60UI2B5MZF9H42,114 Cm Kristal Havuz + 100 Adet 6cm Top + Pompa
15,003857,12li iğne gri
20,003884,2 Adet Nem Alıcı Aparat Rutubet Ve Küf Önleyic...
...,...,...
1550,003013,Çömlek Ayaklı Sırlı Seramik Kedi Maması Kabı 2...
1551,003014,Çömlek Ayaklı Sırlı Seramik Kedi Maması Kabı 2...
1552,gry00018,Özlem Toys Match Up Eşleştirme Kartları 140 Pa...
1562,M-42231,İntex Oyun Havuzu Şişme Taban 100Toplu+Pompa M...


In [285]:
set_origin = set(df_prod_name["model"])
set_new_var = set(df_new_prod["model"])

In [286]:
set_origin.intersection(set_new_var)

set()

In [287]:
# set_new_var.difference(set_origin)

In [288]:
df_prod_name_update = pd.concat([df_prod_name,df_new_prod], axis=0)
df_prod_name_update.reset_index(drop=True, inplace=True)
df_prod_name_update

,model,product_name
0,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö..."
1,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ..."
2,003954,Awox Barmix Çubuk Blender 1000 W
3,004286,Awox Orbit Gizli Rezistanslı Su Isıtıcı Kettle...
4,002656,Sprey Başlıklı Amber Cam 10 Ml Boş - Yedek Şişe
...,...,...
1289,003013,Çömlek Ayaklı Sırlı Seramik Kedi Maması Kabı 2...
1290,003014,Çömlek Ayaklı Sırlı Seramik Kedi Maması Kabı 2...
1291,gry00018,Özlem Toys Match Up Eşleştirme Kartları 140 Pa...
1292,M-42231,İntex Oyun Havuzu Şişme Taban 100Toplu+Pompa M...


In [289]:
df_prod_name_update_duplicated = df_prod_name_update[df_prod_name_update.duplicated(subset="model", keep=False)]
df_prod_name_update_duplicated = df_prod_name_update_duplicated.sort_values(by="model")
df_prod_name_update_duplicated.reset_index(drop=True, inplace=True)
df_prod_name_update_duplicated

,model,product_name
0,001268,Çok Amaçlı Çekmece Içi Akordiyon Organizer Gri...
1,001268,Akordion Çekmece Iç Organizer - L
2,001268,Hueva Home 2'li Akordiyon Organizer L
3,001601,Alas Keçe Kedi ve Köpek Evi Yatağı + Bej Paspas
4,001601,Alas Keçe Kedi ve Köpek Evi Yatağı Gri Paspas ...
...,...,...
171,FURKSOAN,Cool Wheels 5+neon Işıklı
172,TYC065LJ0N170126032824907,Neptün Kaymaz Taban Peluş Kumaş Halı Makinada ...
173,TYC065LJ0N170126032824907,Neptün Kaymaz Taban Peluş Kumaş Halı Makinede ...
174,copy004277,"Kedi Köpek 3 lü Set Parmak Diş Fırçası, Siliko..."


In [290]:
# df_prod_name_update.to_excel(fr"{output_path}\Entegra Aynı Model Tekrarlama.xlsx", index=False)

In [291]:
df_prod_name_update = df_prod_name_update.drop_duplicates(subset="model")
df_prod_name_update.reset_index(drop=True, inplace=True)
df_prod_name_update

,model,product_name
0,003362,"6 Kg Kapatsiteli Styles Gıda Ve Bakliyat, 6 Gö..."
1,001602,"Ahşap Özel İsim Baskılı, Oval Kedi Köpek Mama ..."
2,003954,Awox Barmix Çubuk Blender 1000 W
3,004286,Awox Orbit Gizli Rezistanslı Su Isıtıcı Kettle...
4,002656,Sprey Başlıklı Amber Cam 10 Ml Boş - Yedek Şişe
...,...,...
1193,001265,Çok Amaçlı Çekmece Içi Akordiyon Organizer Gri...
1194,003014,Çömlek Ayaklı Sırlı Seramik Kedi Maması Kabı 2...
1195,gry00018,Özlem Toys Match Up Eşleştirme Kartları 140 Pa...
1196,M-42231,İntex Oyun Havuzu Şişme Taban 100Toplu+Pompa M...


In [292]:
today = date.today()
today

datetime.date(2024, 7, 23)

In [293]:
d1 = today.strftime("%d.%m.%Y")
d1

# # Textual month, day and year	
# d2 = today.strftime("%B %d, %Y")
# print("d2 =", d2)
# 
# # mm/dd/y
# d3 = today.strftime("%m/%d/%y")
# print("d3 =", d3)
# 
# # Month abbreviation, day and year	
# d4 = today.strftime("%b-%d-%Y")
# print("d4 =", d4)

'23.07.2024'

In [294]:
# df_prod_name_update.to_excel(f"C:\\Users\\user\\Desktop\\Data Analysis\\Adoba\\Sales_Report\\Data\\Entegra\\Product Name\\Entegra Model Product Name Update ({d1}).xlsx", index=False)

In [306]:
df_model_in_prod_name = df_prod_name_update[df_prod_name_update["model"].isin(list_no_in_mikro_price)]
df_model_in_prod_name = df_model_in_prod_name.drop_duplicates(subset="model")
df_model_in_prod_name

,model,product_name
3,004286,Awox Orbit Gizli Rezistanslı Su Isıtıcı Kettle...
10,002263,"İksa Devrilmez Ayaklı Portatif Kamp Ocağı, Pik..."
47,001284,Alas Çok Amaçlı Dörtlü Çekmece Düzenleyici Org...
48,002998,Doğal El Yapımı Ardıç Katranlı Sabun Çiftli (M...
49,002581,"Iksa Devrilmez Ayaklı Portatif Kamp Ocağı, Pik..."
...,...,...
1040,002104,Sorgum Unu Glutensi̇z 250 Gr
1041,002374,Da69 Ahşap Resim Şovalesi 37 Cm
1042,002061,Müller Di̇zel Enjektör Temizleyi̇ci̇ (Üstün Al...
1043,002792,21512-21010 Karter Tapasi Kisa Universal
